In [0]:
pip install pytest

Python interpreter will be restarted.
Python interpreter will be restarted.


###Ajustar nome das tabelas caso de erro, referente a existencia delas no diretório quando rodar os testes <br>
De alguma forma o databricks olha para o diretório de uma forma como se existisse algum resquicio de informação, mesmo rodando o código que limpa o diretório

In [0]:
%run
/Users/leonardomares1@gmail.com/CODE_ELEVATE/Diário_de_Bordo/UTILS

In [0]:
import pytest
from unittest.mock import patch

In [0]:
def test_download_csv_with_curl():
    """Testa se a função chama o subprocess corretamente."""
    url = "https://example.com/test.csv"
    local_path = "/tmp/test.csv"

    #Aqui é feito o mock do subprocess para evitar uma chamada real ao cURL
    with patch("subprocess.run") as mock_run:
        download_csv_with_curl(url, local_path)

        # Verifica se subprocess.run foi chamado corretamente
        mock_run.assert_called_once_with(['curl', '-L', url, '-o', local_path], check=True)

    print("✅ Teste `test_download_csv_with_curl` concluído com sucesso!")

test_download_csv_with_curl()

✅ Arquivo baixado para /tmp/test.csv
✅ Teste `test_download_csv_with_curl` concluído com sucesso!


In [0]:
#Vai apagar tudo dentro do diretório, para não ter o erro
dbutils.fs.rm("dbfs:/user/hive/warehouse/teste", recurse=True)

Out[14]: True

In [0]:
nome_tbl = "teste"

def test_salvar_csv_como_delta():
    schema = StructType([
        StructField("DATA_INICIO", StringType(), True),
        StructField("DATA_FIM", StringType(), True),
        StructField("CATEGORIA", StringType(), True),
        StructField("LOCAL_INICIO", StringType(), True),
        StructField("LOCAL_FIM", StringType(), True),
        StructField("PROPOSITO", StringType(), True),
        StructField("DISTANCIA", DecimalType(10, 2), True)
    ])
    caminho_csv = "/tmp/test.csv"
    delimitador = ","

    #Exclui o csv do teste anterior, para roda esse teste
    dbutils.fs.rm("dbfs:/tmp/test.csv", recurse=True)

    #Criando um CSV ficticio
    df_test = spark.createDataFrame([
    ("05-01-2025 10:00", "05-01-2025 12:00", "negócios", "São Paulo", "Rio de Janeiro", "Reunião", 432.5),
    ("05-01-2025 08:00", "05-01-2025 10:00", "pessoal", "São Paulo", "Campinas", "Visita à família", 98.7),
    ("05-02-2025 15:30", "05-02-2025 17:30", "negócios", "Curitiba", "Florianópolis", "Reunião", 300.2),
    ("05-02-2025 09:00", "05-02-2025 11:00", "pessoal", "Brasília", "Goiânia", "Passeio turístico", 209.4),
    ("05-03-2025 13:00", "05-03-2025 15:00", "negócios", "Porto Alegre", "Blumenau", "Reunião", 150.8),
    ("05-03-2025 07:30", "05-03-2025 09:30", "pessoal", "Recife", "João Pessoa", "Visita a amigos", 120.6),
    ("05-04-2025 11:00", "05-04-2025 13:00", "negócios", "Salvador", "Aracaju", "Reunião", 220.9),
    ("05-04-2025 14:30", "05-04-2025 16:30", "pessoal", "Belo Horizonte", "Ouro Preto", None, 100.3),
    ("05-05-2025 16:00", "05-05-2025 18:00", "negócios", "Fortaleza", "Natal", "Reunião", 410.7),
    ("05-05-2025 10:30", "05-05-2025 12:30", "pessoal", "Manaus", "Belém", "Viagem de lazer", 530.1)],
    ["DATA_INICIO", "DATA_FIM", "CATEGORIA", "LOCAL_INICIO", "LOCAL_FIM", "PROPOSITO", "DISTANCIA"])
    df_test.write.option("header", "true").mode("overwrite").csv(caminho_csv)

    df_resultado = salvar_csv_como_delta(schema, caminho_csv, nome_tbl, delimitador)
    assert spark._jsparkSession.catalog().tableExists(nome_tbl), "A Tabela não foi criada"

    print("✅ Teste salvar csv como delta foi bem sucedido!")

def test_salvar_csv_como_delta_schema():

    df_resultado = spark.sql(f"SELECT * FROM {nome_tbl}")

    assert df_resultado.count() > 0, "A Tabela esta vazia"

    schema_esperado = ["DATA_INICIO", "DATA_FIM", "CATEGORIA", "LOCAL_INICIO", "LOCAL_FIM", "PROPOSITO", "DISTANCIA"]
    schema_obtido = df_resultado.schema.names

    assert schema_esperado == schema_obtido, (f"Schemas não correspondem. Esperado: {schema_esperado} x Obtido: {schema_obtido}")
    
    print("✅ Teste de schema foi bem sucedido!")

test_salvar_csv_como_delta()
test_salvar_csv_como_delta_schema()

✅ Tabela Delta 'teste' criada com sucesso!
✅ Teste salvar csv como delta foi bem sucedido!
✅ Teste de schema foi bem sucedido!


In [0]:
def test_ler_tabela_delta():

    df = ler_tabela_delta(nome_tbl)

    assert df_resultado is not None, "O Dataframe é None"

    assert df_resultado is not None, "O Dataframe é None"
    assert df.count() > 0, "O Dataframe está vasio"

    print("Teste de leitura de tabela")

In [0]:
#display(dbutils.fs.ls("dbfs:/user/hive/warehouse/tabela_teste"))

In [0]:
#Vai apagar tudo dentro do diretório, para não ter o erro
dbutils.fs.rm("dbfs:/user/hive/warehouse/tabela_teste", recurse=True)

In [0]:
colunas = [
    "DT_REFE date",
    "QT_CORR int",
    "QT_CORR_NEG int",
    "QT_CORR_PESS int",
    "VL_MAX_DIST int",
    "VL_MIN_DIST int",
    "VL_AVG_DIST decimal (5,2)",
    "QT_CORR_REUNI int",
    "QT_CORR_NAO_REUNI int"]
nome_tbl = "tabela_teste_v2"
particao = "DT_REFE"


def test_criar_tabela_calculada():
    
    df_resultado = criar_tabela_calculada(colunas, nome_tbl, particao)
    assert df_resultado is not None, "Erro: O Dataframe retornado é None!"
    
    print("✅ Teste de criação concluido com sucesso!")

def test_criar_tabela_calculada_coluna():

    df_test = spark.sql(f"select * from {nome_tbl}")

    # Extrai apenas os nomes das colunas
    colunas_esperadas = [col.split()[0] for col in colunas]
   
    assert set(df_test.columns) == set(colunas_esperadas), "Colunas não correspondem as esperadas"
    # assert df_test.count() > 0, "O Dataframe esta vasio!"
    print("✅ Teste de colunas concluido com sucesso!")

test_criar_tabela_calculada()
test_criar_tabela_calculada_coluna()


✅ Tabela tabela_teste_v2, criada com sucesso
✅ Teste de criação concluido com sucesso!
✅ Teste de colunas concluido com sucesso!


In [0]:
def test_tratar_data():
    colunas = [
        "*",
        "CAST(TO_TIMESTAMP(DATA_INICIO, 'MM-dd-yyyy HH:mm') AS DATE) AS DT_REFE"
        ]
    tempview = "teste_data"

    df_resultado = tratar_data(colunas, "teste", tempview)
    assert df_resultado is not None, "Dataframe retornado é None!"

    df_test = spark.sql(f'select * from {tempview}')
    assert df_test.count() > 0, "df_test está vazio"

    colunas_esperadas = ["DATA_INICIO", "DATA_FIM", "CATEGORIA", "LOCAL_INICIO", "LOCAL_FIM", "PROPOSITO", "DISTANCIA", "DT_REFE"]
    assert set(df_test.columns) == set(colunas_esperadas), "As colunas não são correspondentes"
    

    print("✅ O Teste tratar_data foi bem-sucedido!!!")

test_tratar_data()

✅ O Tratamento de data foi bem-sucedido!!!
✅ O Teste tratar_data foi bem-sucedido!!!


In [0]:
%sql
select * from teste_data

DATA_INICIO,DATA_FIM,CATEGORIA,LOCAL_INICIO,LOCAL_FIM,PROPOSITO,DISTANCIA,DT_REFE
05-01-2025 08:00,05-01-2025 10:00,pessoal,São Paulo,Campinas,Visita à família,98.70,2025-05-01
05-01-2025 10:00,05-01-2025 12:00,negócios,São Paulo,Rio de Janeiro,Reunião,432.50,2025-05-01
05-03-2025 07:30,05-03-2025 09:30,pessoal,Recife,João Pessoa,Visita a amigos,120.60,2025-05-03
05-02-2025 15:30,05-02-2025 17:30,negócios,Curitiba,Florianópolis,Reunião,300.20,2025-05-02
05-04-2025 11:00,05-04-2025 13:00,negócios,Salvador,Aracaju,Reunião,220.90,2025-05-04
05-04-2025 14:30,05-04-2025 16:30,pessoal,Belo Horizonte,Ouro Preto,null,100.30,2025-05-04
05-02-2025 09:00,05-02-2025 11:00,pessoal,Brasília,Goiânia,Passeio turístico,209.40,2025-05-02
05-03-2025 13:00,05-03-2025 15:00,negócios,Porto Alegre,Blumenau,Reunião,150.80,2025-05-03
05-05-2025 16:00,05-05-2025 18:00,negócios,Fortaleza,Natal,Reunião,410.70,2025-05-05
05-05-2025 10:30,05-05-2025 12:30,pessoal,Manaus,Belém,Viagem de lazer,530.10,2025-05-05


In [0]:
def test_count_categoria():
    filtro_tbl = "group by DT_REFE, DISTANCIA, CATEGORIA"
    colunas = [
        "DT_REFE",
        "DISTANCIA",
        "1 as qtd",
        "CASE WHEN CATEGORIA = 'negócios' THEN 1 ELSE 0 END AS qtd_neg",
        "CASE WHEN CATEGORIA = 'pessoal' THEN 1 ELSE 0 END AS qtd_pes"
    ]
    tempview = "teste_categ"

    df_resultado = count_categoria(colunas, "teste_data", tempview, filtro_tbl)
    assert df_resultado is not None, "Dataframe retornado é None!"

    df_test = spark.sql(f'select * from {tempview}')
    assert df_test.count() > 0, "df_test está vazio"

    colunas_esperadas = ["DT_REFE", "DISTANCIA", "qtd", "qtd_neg", "qtd_pes"]
    assert set(df_test.columns) == set(colunas_esperadas), "As colunas não são correspondentes"
    

    print("✅ O Teste count_categoria foi bem-sucedido!!!")

test_count_categoria()
    

✅ A contagem de categoria foi bem-sucedido!!!
✅ O Teste count_categoria foi bem-sucedido!!!


In [0]:
%sql
select * from teste_categ

DT_REFE,DISTANCIA,qtd,qtd_neg,qtd_pes
2025-05-01,98.70,1,0,1
2025-05-01,432.50,1,1,0
2025-05-03,120.60,1,0,1
2025-05-02,300.20,1,1,0
2025-05-04,220.90,1,1,0
2025-05-04,100.30,1,0,1
2025-05-02,209.40,1,0,1
2025-05-03,150.80,1,1,0
2025-05-05,410.70,1,1,0
2025-05-05,530.10,1,0,1


In [0]:
filtro_tbl = "group by DT_REFE"
def test_categoria_agregada():
    tempview = "test_categ_agreg"
    colunas = [
        "DT_REFE",
        "sum(qtd) as QT_CORR",
        "sum(qtd_neg) as QT_CORR_NEG",
        "sum(qtd_pes) as QT_CORR_PESS",
        "max(DISTANCIA) as VL_MAX_DIST",
        "min(DISTANCIA) as VL_MIN_DIST",
        "avg(DISTANCIA) as VL_AVG_DIST"
    ]

    df_resultado = count_categoria(colunas, "teste_categ", tempview, filtro_tbl)
    assert df_resultado is not None, "Dataframe retornado é None!"

    df_test = spark.sql(f'select * from {tempview}')
    assert df_test.count() > 0, "df_test está vazio"

    colunas_esperadas = ["DT_REFE", "QT_CORR", "QT_CORR_NEG", "QT_CORR_PESS", "VL_MAX_DIST", "VL_MIN_DIST", "VL_AVG_DIST"]
    assert set(df_test.columns) == set(colunas_esperadas), "As colunas não são correspondentes"
    

    print("✅ O Teste count_categoria foi bem-sucedido!!!")

test_categoria_agregada()


✅ A contagem de categoria foi bem-sucedido!!!
✅ O Teste count_categoria foi bem-sucedido!!!


In [0]:
%sql
select * from test_categ_agreg

DT_REFE,QT_CORR,QT_CORR_NEG,QT_CORR_PESS,VL_MAX_DIST,VL_MIN_DIST,VL_AVG_DIST
2025-05-02,2,1,1,300.20,209.40,254.800000
2025-05-04,2,1,1,220.90,100.30,160.600000
2025-05-03,2,1,1,150.80,120.60,135.700000
2025-05-05,2,1,1,530.10,410.70,470.400000
2025-05-01,2,1,1,432.50,98.70,265.600000


In [0]:
def test_count_proposito():
    colunas = [
        "DT_REFE",
        "sum(CASE WHEN PROPOSITO = 'Reunião' THEN 1 ELSE 0 END) AS QT_CORR_REUNI",
        "sum(CASE WHEN PROPOSITO IS NOT NULL AND PROPOSITO <> 'Reunião' THEN 1 ELSE 0 END) AS QT_CORR_NAO_REUNI"
    ]
    tempview = "teste_prop"

    df_resultado = count_proposito(colunas, "teste_data", tempview, filtro_tbl)
    assert df_resultado is not None, "Dataframe retornado é None!"

    df_test = spark.sql(f'select * from {tempview}')
    assert df_test.count() > 0, "df_test está vazio"

    colunas_esperadas = ["DT_REFE", "QT_CORR_REUNI", "QT_CORR_NAO_REUNI"]
    assert set(df_test.columns) == set(colunas_esperadas), "As colunas não são correspondentes"
    

    print("✅ O Teste count_proposito foi bem-sucedido!!!")

test_count_proposito()

✅ A contagem de proposito foi bem-sucedido!!!
✅ O Teste count_proposito foi bem-sucedido!!!


In [0]:
%sql
select * from teste_prop

DT_REFE,QT_CORR_REUNI,QT_CORR_NAO_REUNI
2025-05-01,1,1
2025-05-03,1,1
2025-05-02,1,1
2025-05-04,1,0
2025-05-05,1,1


In [0]:
def test_join_tabelas():
    colunas = [
        "c.DT_REFE",
        "c.QT_CORR",
        "c.QT_CORR_NEG",
        "c.QT_CORR_PESS",
        "c.VL_MAX_DIST",
        "c.VL_MIN_DIST",
        "c.VL_AVG_DIST",
        "p.QT_CORR_REUNI",
        "p.QT_CORR_NAO_REUNI"
    ]
    tbl_filtro = "test_categ_agreg c"
    tbl_join = "teste_prop p"
    on = "c.DT_REFE = p.DT_REFE"
    filtro_tbl = """
    order by c.DT_REFE asc
    """
    tempview = "teste_tbl_final"

    df_resultado = join_tabelas(colunas, tbl_filtro, tbl_join, on, filtro_tbl, tempview)
    assert df_resultado is not None, "Dataframe retornado é None!"
    assert "QT_CORR_REUNI" in df_resultado.columns, "A coluna QT_CORR_REUNI, não foi encontrada"

    df_test = spark.sql(f'select * from {tempview}')
    assert df_test.count() > 0, "df_test está vazio"

    assert df_resultado.count() == df_test.count(), "A contangem não bate"

    print("✅ O Teste join_tabelas foi bem-sucedido!!!")

test_join_tabelas()

✅ O join das contagens foi bem-sucedido!!!
✅ O Teste join_tabelas foi bem-sucedido!!!


In [0]:
%sql
select * from teste_tbl_final

DT_REFE,QT_CORR,QT_CORR_NEG,QT_CORR_PESS,VL_MAX_DIST,VL_MIN_DIST,VL_AVG_DIST,QT_CORR_REUNI,QT_CORR_NAO_REUNI
2025-05-01,2,1,1,432.50,98.70,265.600000,1,1
2025-05-02,2,1,1,300.20,209.40,254.800000,1,1
2025-05-03,2,1,1,150.80,120.60,135.700000,1,1
2025-05-04,2,1,1,220.90,100.30,160.600000,1,0
2025-05-05,2,1,1,530.10,410.70,470.400000,1,1


In [0]:
def test_ingestao_dados():
    tbl_calc = "tabela_teste_v2"
    tbl_join = "teste_tbl_final"

    df_resultado = ingestao_dados(tbl_calc, tbl_join)
    df_test = spark.sql(f"select * from {tbl_join}")

    assert df_resultado.count() == df_test.count(), "A contagem das tabelas não bate"

    print("✅ O Teste ingestão de dados foi bem-sucedido!!!")

test_ingestao_dados()

✅ A ingestão foi bem-sucedida!!!
✅ O Teste ingestão de dados foi bem-sucedido!!!


In [0]:
%sql
select * from tabela_teste_v2

DT_REFE,QT_CORR,QT_CORR_NEG,QT_CORR_PESS,VL_MAX_DIST,VL_MIN_DIST,VL_AVG_DIST,QT_CORR_REUNI,QT_CORR_NAO_REUNI
2025-05-05,2,1,1,530,410,470.40,1,1
2025-05-04,2,1,1,220,100,160.60,1,0
2025-05-03,2,1,1,150,120,135.70,1,1
2025-05-02,2,1,1,300,209,254.80,1,1
2025-05-01,2,1,1,432,98,265.60,1,1


#Teste em SQL

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/teste_sql", recurse=True)

Out[32]: True

In [0]:
spark.sql(
    f"""
    CREATE OR REPLACE TABLE teste_sql (
    DATA_INICIO STRING,
    DATA_FIM STRING,
    CATEGORIA STRING,
    LOCAL_INICIO STRING,
    LOCAL_FIM STRING,
    PROPOSITO STRING,
    DISTANCIA DECIMAL(10,2))
    USING DELTA
""")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1558279804100179>:1
----> 1 spark.sql(
      2     f"""
      3     CREATE OR REPLACE TABLE teste_sql (
      4     DATA_INICIO STRING,
      5     DATA_FIM STRING,
      6     CATEGORIA STRING,
      7     LOCAL_INICIO STRING,
      8     LOCAL_FIM STRING,
      9     PROPOSITO STRING,
     10     DISTANCIA DECIMAL(10,2))
     11     USING DELTA
     12 """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, arg

In [0]:
 %sql
INSERT INTO teste_sql (DATA_INICIO, DATA_FIM, CATEGORIA, LOCAL_INICIO, LOCAL_FIM, PROPOSITO, DISTANCIA) VALUES
('05-01-2025 10:00', '05-01-2025 12:00', 'negócios', 'São Paulo', 'Rio de Janeiro', 'Reunião', 432.5),
('05-01-2025 08:00', '05-01-2025 10:00', 'pessoal', 'São Paulo', 'Campinas', 'Visita à família', 98.7),
('05-02-2025 15:30', '05-02-2025 17:30', 'negócios', 'Curitiba', 'Florianópolis', 'Reunião', 300.2),
('05-02-2025 09:00', '05-02-2025 11:00', 'pessoal', 'Brasília', 'Goiânia', 'Passeio turístico', 209.4),
('05-03-2025 13:00', '05-03-2025 15:00', 'negócios', 'Porto Alegre', 'Blumenau', 'Reunião', 150.8),
('05-03-2025 07:30', '05-03-2025 09:30', 'pessoal', 'Recife', 'João Pessoa', 'Visita a amigos', 120.6),
('05-04-2025 11:00', '05-04-2025 13:00', 'negócios', 'Salvador', 'Aracaju', 'Reunião', 220.9),
('05-04-2025 14:30', '05-04-2025 16:30', 'pessoal', 'Belo Horizonte', 'Ouro Preto', NULL, 100.3),
('05-05-2025 16:00', '05-05-2025 18:00', 'negócios', 'Fortaleza', 'Natal', 'Reunião', 410.7),
('05-05-2025 10:30', '05-05-2025 12:30', 'pessoal', 'Manaus', 'Belém', 'Viagem de lazer', 530.1),
('05-06-2025 08:00', '05-06-2025 10:00', 'negócios', 'São Paulo', 'Brasília', 'Reunião', 850.6),
('05-06-2025 18:00', '05-06-2025 20:00', 'pessoal', 'Curitiba', 'Joinville', NULL, 130.9),
('05-07-2025 14:00', '05-07-2025 16:00', 'negócios', 'Vitória', 'Rio de Janeiro', 'Reunião', 400.5),
('05-07-2025 12:00', '05-07-2025 14:00', 'pessoal', 'São Luís', 'Teresina', 'Evento cultural', 250.8),
('05-08-2025 17:30', '05-08-2025 19:30', 'negócios', 'Belém', 'Macapá', 'Reunião', 320.2),
('05-08-2025 09:30', '05-08-2025 11:30', 'pessoal', 'Campo Grande', 'Dourados', 'Encontro com amigos', 150.4),
('05-09-2025 20:00', '05-09-2025 22:00', 'negócios', 'Florianópolis', 'Porto Alegre', 'Reunião', 470.7),
('05-09-2025 07:00', '05-09-2025 09:00', 'pessoal', 'Natal', 'Recife', NULL, 300.9),
('05-10-2025 11:30', '05-10-2025 13:30', 'negócios', 'Aracaju', 'Maceió', 'Reunião', 220.1),
('05-10-2025 15:00', '05-10-2025 17:00', 'pessoal', 'Rio Branco', 'Porto Velho', 'Viagem de descanso', 350.4),
('05-11-2025 08:30', '05-11-2025 10:30', 'negócios', 'João Pessoa', 'Natal', 'Reunião', 180.2),
('05-11-2025 16:30', '05-11-2025 18:30', 'pessoal', 'Maceió', 'Salvador', 'Férias', 700.3),
('05-12-2025 10:00', '05-12-2025 12:00', 'negócios', 'Brasília', 'São Paulo', 'Reunião', 870.5),
('05-12-2025 13:30', '05-12-2025 15:30', 'pessoal', 'Porto Alegre', 'Curitiba', NULL, 450.6);

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1032241395436840>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-1032241395436840>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("SU5TRVJUIElOVE8gdGVzdGVfc3FsIChEQVRBX0lOSUNJTywgREFUQV9GSU0sIENBVEVHT1JJQSwgTE9DQUxfSU5JQ0lPLCBMT0NBTF9GSU0sIFBST1BPU0lUTywgRElTVEFOQ0lBKSBWQUxVRVMKKCcwNS0wMS0yMDI1IDEwOjAwJywgJzA1LTAxLTIwMjUgMTI6MDAnLCAnbmVnw7NjaW9zJywgJ1PDo28gUGF1bG8nLCAnUmlvIGRlIEphbmVpcm8nLCAnUmV1bmnDo28nLCA0MzIuNSksCignMDUtMDEtMjAyNSAwODowMCcsICcwNS0wMS0yMDI1IDEwOjAwJywgJ3Blc3NvYWwnLCAnU8OjbyBQYXVsbycsICdDYW1waW5hcycsICdWaXNpdGEgw6AgZmFtw61saWEnLCA5OC43KSwKKCcwNS0wMi0yMDI1IDE1OjMwJywgJzA1LTAyLTIwMjUg

In [0]:
#Vai apagar tudo dentro do diretório, para não ter o erro 
dbutils.fs.rm("dbfs:/user/hive/warehouse/sql_final", recurse=True)

Out[35]: True

In [0]:
spark.sql(
    f"""
    create or replace table sql_final(
        DT_REFE date,
        QT_CORR int,
        QT_CORR_NEG int,
        QT_CORR_PESS int,
        VL_MAX_DIST int,
        VL_MIN_DIST int,
        VL_AVG_DIST double,
        QT_CORR_REUNI int,
        QT_CORR_NAO_REUNI int

    )
""")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1032241395436841>:1
----> 1 spark.sql(
      2     f"""
      3     create or replace table sql_final(
      4         DT_REFE date,
      5         QT_CORR int,
      6         QT_CORR_NEG int,
      7         QT_CORR_PESS int,
      8         VL_MAX_DIST int,
      9         VL_MIN_DIST int,
     10         VL_AVG_DIST double,
     11         QT_CORR_REUNI int,
     12         QT_CORR_NAO_REUNI int
     13 
     14     )
     15 """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spa

In [0]:
%sql
insert into sql_final 
WITH contagem as
(SELECT 
  CAST(TO_TIMESTAMP(DATA_INICIO, 'MM-dd-yyyy HH:mm') AS DATE) AS DT_REFE,
  count(*) as qtd,
  CASE WHEN CATEGORIA = 'Negocio' THEN COUNT(categoria) else 0 END AS  qt_corrida_negocio,
  CASE WHEN CATEGORIA = 'Pessoal' THEN COUNT(categoria) else 0 END AS  qt_corrida_pessoal,
  DISTANCIA,
  CASE WHEN PROPOSITO = 'Reunião' then count(PROPOSITO) end as reuniao,
  CASE WHEN PROPOSITO IS NOT NULL AND PROPOSITO <> 'Reunião' THEN count(PROPOSITO) else 0 END AS pessoal 
FROM info_transportes
GROUP BY DATA_INICIO, CATEGORIA, DISTANCIA, PROPOSITO

ORDER BY DATA_INICIO ASC)

SELECT
  dt_refe,
  sum(qtd) as QT_CORR,
  sum(qt_corrida_negocio) as QT_CORR_NEG,
  sum(qt_corrida_pessoal) as QT_CORR_PESS,
  max(DISTANCIA) as VL_MAX_DIST,
  min(DISTANCIA) as VL_MIN_DIST,
  avg(DISTANCIA) as VL_AVG_DIST,
  sum(reuniao) as QT_CORR_REUNI,
  sum(pessoal) as QT_CORR_NAO_REUNI
  


FROM contagem
GROUP BY DT_REFE

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1558279804100181>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-1558279804100181>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("aW5zZXJ0IGludG8gc3FsX2ZpbmFsIApXSVRIIGNvbnRhZ2VtIGFzCihTRUxFQ1QgCiAgQ0FTVChUT19USU1FU1RBTVAoREFUQV9JTklDSU8sICdNTS1kZC15eXl5IEhIOm1tJykgQVMgREFURSkgQVMgRFRfUkVGRSwKICBjb3VudCgqKSBhcyBxdGQsCiAgQ0FTRSBXSEVOIENBVEVHT1JJQSA9ICdOZWdvY2lvJyBUSEVOIENPVU5UKGNhdGVnb3JpYSkgZWxzZSAwIEVORCBBUyAgcXRfY29ycmlkYV9uZWdvY2lvLAogIENBU0UgV0hFTiBDQVRFR09SSUEgPSAnUGVzc29hbCcgVEhFTiBDT1VOVChjYXRlZ29yaWEpIGVsc2UgMCBFTkQgQVMgIHF0X2NvcnJpZGFfcGVzc29hbCwKICBESVNUQU5DSUEsCiAgQ0FTRSBXSEVOIFBST1BPU0lU

In [0]:
%sql
create or replace temp view init as
select
*,
CAST(TO_TIMESTAMP(DATA_INICIO, 'MM-dd-yyyy HH:mm') AS DATE) AS DT_REFE
from info_transportes

In [0]:
%sql
SELECT * FROM INIT

In [0]:
%sql
create or replace temp view fase_2 as
select
  DT_REFE,
  DISTANCIA,
  1 AS QTD,
  (CASE WHEN CATEGORIA = 'Negocio' THEN 1 else 0 END) AS  qt_corrida_negocio,
  (CASE WHEN CATEGORIA = 'Pessoal' THEN 1 else 0 END) AS  qt_corrida_pessoal
from init

In [0]:
%sql
select * from fase_2

In [0]:
%sql
create or replace temp view fase_3 as
select
  DT_REFE,
  SUM(CASE WHEN PROPOSITO = 'Reunião' then 1 end) as reuniao,
  SUM(CASE WHEN PROPOSITO IS NOT NULL AND PROPOSITO <> 'Reunião' THEN 1 else 0 END) AS pessoal
from init
GROUP BY 1

In [0]:
%sql
select * from fase_3

In [0]:
%sql
select
c.DT_REFE,
sum(c.QTD),
sum(c.qt_corrida_negocio),
sum(c.qt_corrida_pessoal),
maX(c.DISTANCIA),
min(c.DISTANCIA),
avg(c.DISTANCIA),
sum(p.reuniao),
sum(p.pessoal)
from fase_2 c
join fase_3 p
on c.DT_REFE = p.DT_REFE
group by c.DT_REFE, p.dt_refe
order by c.DT_REFE desc

In [0]:
%sql
select * from sql_final